# Redis tutorial

## Prerequisites

In [3]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 8.5 MB/s eta 0:00:00


In [4]:
import redis

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* 
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

#host='redis' spécifie l'hôte du serveur redis auquel la connexion doit être rétablie.

In [5]:
r = redis.Redis(host='redis', port=6379)

In [4]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [5]:
r.set("greetings", "hello world")

True

In [6]:
value1 = r.get('greetings')
print(value1)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [7]:
r.set("connections", "0")

True

In [8]:
r.incr("connections")

1

In [9]:
r.get("connections")

b'1'

### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [14]:
r.rpush('colors','red','black','blue')

3

**Q** : Display the second element of the list with `LINDEX`

In [21]:
r.lindex('colors',1)

b'black'

**Q** : Display the full list with `LRANGE`

In [24]:
r.lrange('colors',0,2)

[b'red', b'black', b'blue']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [25]:
r.sadd('nosql','redis','mongodb','orientdb')

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [26]:
r.sismember('nosql', 'mysql')

0

#0 car 'mysql' n'existe pas dans 'nosql'.

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [27]:
r.sadd('nosql', 'hbase')

1

In [31]:
r.smembers('nosql')

{b'hbase', b'mongodb', b'orientdb', b'redis'}

In [32]:
r.sadd('nosql', 'redis')

0

In [34]:
r.smembers('nosql')

{b'hbase', b'mongodb', b'orientdb', b'redis'}

#ajout de 'rdis' à nouveau ne marche pas.

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [50]:
r.zadd("top14",{'Agen':10, 'Bordeaux':33, 'Brive':32, 'Castres':29, 'Clermont':38, 'Grenoble':24, 'La Rochelle':26, 'Montpellier':32, 'Oyonnax':14, 'Pau':20, 'Racing':40, 'Stade Français': 22, 'Toulon':36, 'Toulouse':36})

12

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [52]:
r.zscore("top14","Toulon")

36.0

In [60]:
r.zrank("top14","Toulon",withscore=True)

[10, b'36']

#Toulon est la 10 ème ville (en partant de 0) avec le score le plus élevé, lorsque les scores sont triés du plus petit au plus grand.

**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [62]:
r.zrevrank("top14","Toulon", withscore=True)

[3, b'36']

#Toulon est la 3ème ville avec le score le plus élevé (égalité avec Toulouse) lorsque les scores sont triés du plus élevé vers le plus petit.

**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [64]:
#the 3 best teams:
r.zrevrange("top14", 0 ,2, withscores=True)

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]

In [68]:
#teams with more than 35 points:
r.zrevrangebyscore("top14", 40, 36, withscores=True)

[(b'Racing', 40.0),
 (b'Clermont', 38.0),
 (b'Toulouse', 36.0),
 (b'Toulon', 36.0)]

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [8]:
r.hset("user:1", mapping={"id":1,"name":"Jean","age":22})

0

In [9]:
r.hget("user:1", "name")

b'Jean'

In [10]:
r.hgetall("user:1")

{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [12]:
r.hsetnx("user:1", key="city", value="Lyon")
r.hgetall("user:1")

{b'id': b'1', b'name': b'Jean', b'age': b'22', b'city': b'Lyon'}

In [15]:
r.hset("user:1", key="name", value="Paul")

0

In [16]:
r.hgetall("user:1")

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [ ]:
!pip install requests

In [ ]:
!pip install requests_cache

In [19]:
import requests_cache
import requests

In [99]:
#response=requests.put('http://my-api.fr/user/10', json= {"id": 10, "name": "jean"})

In [31]:
redis = {
    "api.com/jean": {"id": 10, "name": "jean"},
    "api.com/marie":  {"id": 11, "name": "marie"},
}

In [32]:
redis.get("api.com/jean")

{'id': 10, 'name': 'jean'}

In [34]:
r.hset('put-http://my-api.fr/user/10', mapping={"id": 10, "name": "jean"})

0

In [35]:
r.hgetall('put-http://my-api.fr/user/10')

{b'id': b'10', b'name': b'jean'}

In [36]:
r.hset('get-http://my-api.fr/user/10', mapping={"id": 12, "name": "paul"})

0

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

**Q** : Create a set of cache request keys.

In [40]:
# Set of cache request keys
cache_request_keys = {'put-http://my-api.fr/user/10','get-http://my-api.fr/user/10'}

In [41]:
# Add keys to the Redis set
redis_key = 'cache_request_keys'
r.sadd(redis_key, *cache_request_keys)

0

In [42]:
# Retrieve and print the set of cache request keys
cached_keys = r.smembers(redis_key)
cached_keys= {key.decode('utf-8') for key in cached_keys}
print("Cached Request Keys:", cached_keys)

Cached Request Keys: {'put-http://my-api.fr/user/10', 'get-http://my-api.fr/user/10'}


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [45]:
get_request_key = 'get-http://my-api.fr/user/10'
if get_request_key in cached_keys:
    print("GET request exists in cache.")
else:
    print("None")

GET request exists in cache.


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [47]:
put_request_key = 'put-http://my-api.fr/user/10'
r.srem(redis_key, put_request_key)
print("PUT request has been removed from cache.")

PUT request has been removed from cache.


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [ ]:
!redis-cli flushall